In [1]:
#import libraries
import boto3
import csv
import json
import time

In [2]:
# Connection to AWS
folder = '/home/propietari/Documents/claus/' # TODO: change to your local path
file_name = 'AWS_S3_keys_wri.json' # TODO: Change to your filename
# folder = 'C:/Users/jordi/Documents/claus/' # TODO: change to your local path
# file_name = 'AWS_S3_keys_wri.json' # TODO: Change to your filename
file = folder + file_name

with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
# s3BucketName = "wri-testing"
s3BucketName = "wri-nlp-policy"
# region = 'eu-central-1'
region = "us-east-1"

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [8]:
prefix = "english_documents/raw_pdf/"
i = 0
for object in s3.Bucket(s3BucketName).objects.all().filter(Prefix=prefix):
    srcKey = object.key
    if ".txt" in srcKey:
        print(i, end = "\r")
        newKey = srcKey.replace(".txt", ".pdf")
        copySource = s3BucketName + '/' + srcKey         
        s3.Object(s3BucketName, newKey).copy_from(CopySource=copySource)
        s3.Object(s3BucketName, srcKey).delete()
    i += 1

3
5
6
8
9
11
12
13
14
15
17
20
22
23
25
26
27
28
29
30
31
38
39
41
42
43
44
45
46
47
48
49
50
54
57
58
59
61
62
63
69
73
75
76
77
80
81
82
83
84
85
86
87
88
89
95
97
98
101
103
104
106
107
108
110
111
112
113
114
115
116
118
119
120
123
124
126
128
131
135
136
138
139
140
142
143
144
146
150
151
155
163
165
166
168
170
171
174
175
180
181
184
185
186
188
189
191
192
194
195
197
198
199
201
202
203
204
205
206
209
210
213
217
221
222
224
228
229
231
236
237
239
240
243
244
245
246
247
250
251
252
256
257
260
266
267
268
269
270
271
273
274
276
279
280
281
283
284
285
286
291
292
294
300
301
302
304
306
307
309
312
313
315
316
319
320
322
323
324
325
326
327
328
331
332
333
335
336
337
339
340
341
342
345
346
347
352
354
356
358
359
361
363
368
369
370
373
374
376
377
378
379
380
382
386
387
390
393
399
400
404
405
406
407
408
409
410
411
412
413
418
419
420
421
422
425
426
427
429
431
432
434
435
438
439
440
442
443
444
449
453
455
456
457
458
460
464
469
470
472
473
474
475
476
477
480

In [3]:
def startJob(s3BucketName, objectName):
    response = None
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.start_document_text_detection(DocumentLocation={
        'S3Object': {'Bucket': s3BucketName,
                     'Name': objectName
                    }
        })
    return response["JobId"]
def isJobComplete(jobId):
 # For production use cases, use SNS based notification 
 # Details at: https://docs.aws.amazon.com/textract/latest/dg/api-async.html
    time.sleep(5)
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.get_document_text_detection(JobId=jobId)
    status = response["JobStatus"]
#     print("Job status: {}".format(status))
    while(status == "IN_PROGRESS"):
        time.sleep(5)
        response = client.get_document_text_detection(JobId=jobId)
        status = response["JobStatus"]
#         print("Job status: {}".format(status))
    return status
def getJobResults(jobId):
    pages = []
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.get_document_text_detection(JobId=jobId)
    
    pages.append(response)
#     print("Resultset page recieved: {}".format(len(pages)))
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
        while(nextToken):
            response = client.get_document_text_detection(JobId=jobId, NextToken=nextToken)
            pages.append(response)
#             print("Resultset page recieved: {}".format(len(pages)))
            nextToken = None
            if('NextToken' in response):
                nextToken = response['NextToken']
    return pages

def save_txt_file(file, file_name):
    f = open(file_name, "w")
    f.write(file)
    f.close()

In [4]:
def AWS_pdf_to_text(key, s3BucketName):
    # Document
    documentName = key
    jobId = startJob(s3BucketName, documentName)
#     print("Started job with id: {}".format(jobId))
    if(isJobComplete(jobId)):
        response = getJobResults(jobId)
    #print(response)
    # Print detected text
    policy = ""
    for resultPage in response:
#         print(resultPage)
        try:
            for item in resultPage["Blocks"]:
                if item["BlockType"] == "LINE":
                    policy = policy + item["Text"] + "\n"
#                     print ('\033[94m' + item["Text"] + '\033[0m')
        except:
            f = open("../output/log_file.txt", 'a')
            f.write(f"The file {key} could not be converted\n")
            f.close()
            
            print(f"The file {key} could not be converted")
    return policy

In [5]:
keyword = "0bc17448527c7798e84937eb897bdaa82c3c4a3c.pdf"
prefix = "spanish_documents/raw_pdf_updated/"
i = 0

for obj in s3.Bucket(s3BucketName).objects.all().filter(Prefix = prefix):#.:
    if ".pdf" in obj.key:# and i > 48: lower_limit < i < upper_limit:
#         print(i, "**", obj.key)
        print(i, end = "\r")
        key = obj.key.replace("raw_pdf_updated", "text_files/HSSC/new").replace(".pdf", ".txt")
        policy = AWS_pdf_to_text(obj.key, s3BucketName)
        if policy != "":
            s3.Object(s3BucketName, key).put(Body = policy)
#         save_txt_file(policy, "/home/propietari/Documents/S3/" + obj.key[:-4] + ".txt")
    i += 1

1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	33	34	35	36	37	38	39	40	41	42	43	44	45	46	47	48	49	50	51	52	53	54	55	56	57	58	59	60	61	62	63	64	65	66	67	68	69	70	71	72	73	74	75	76	77	78	79	80	81	82	83	84	85	86	87	88	89	90	91	92	93	94	95	96	97	98	99	100	101	102	103	104	105	106	107	108	109	110	111	112	113	114	115	116	117	118	119	120	121	122	123	124	125	126	127	128	129	130	131	132	133	134	135	136	137	138	139	140	141	142	143	144	145	146	147	148	149	150	151	152	153	154	155	156	157	158	159	160	161	162	163	164	165	166	167	168	169	170	171	172	173	174	175	176	177	178	179	180	181	182	183	184	185	186	187	188	189	190	191	192	193	194	195	196	197	198	199	200	201	202	203	204	205	206	207	208	209	210	211	212	213	214	215	216	217	218	219	220	221	222	223	224	225	226	227	228	229	230	231	232	233	234	235	236	

In [ ]:
policy